In [2]:
import pandas as pd
import csv
import os
import time
from collections import defaultdict
import numpy as np
import itertools
import db_structure
import networkx as nx
import matplotlib.pyplot as plt
import utilities as u
from flask import jsonify
import logging
# import exceptions
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from web.models import *
from web import db
import graph

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
%load_ext autoreload
%aimport -web
%autoreload 2

In [ ]:
dataset_name = 'SCA_AcuteCare'
draw_db = db_structure.DBExtractor(dataset_name)

G = nx.DiGraph()
table_names = [x.table_name for x in db.session.query(TableMetadata).filter(TableMetadata.dataset_name == dataset_name, TableMetadata.num_records >= 10000).all()]
G.add_nodes_from(table_names)
for table_name in table_names:
    for sibling in draw_db.find_table_siblings(table_name):
        if sibling in table_name:
            G.add_edge(table_name, sibling)
            G.add_edge(sibling, table_name)
    for child in draw_db.find_table_children(table_name):
        if child in table_names:
            G.add_edge(table_name, child)
plt.figure(figsize=(10,10))

#draw with networkx built-ins
nx.draw_networkx(G, node_shape="None", width=0.2)

#OR

#draw using pygraphviz
#A = nx.nx_agraph.to_agraph(G)
#H = nx.nx_agraph.from_agraph(A)
#nx.draw_spring(H, node_shape="None", with_labels=True, width=0.2)

In [6]:
x = db_structure.DBMaker(dataset_name='TOPICC', directory_path='datasets/TOPICC')
x.create_db_metadata(dump_to_data_db=True)

In [5]:
db_structure.DBDestroyer('TOPICC').remove_db()

In [7]:
y = db_structure.DBLinker(dataset_name='TOPICC')

In [8]:
y.add_global_fk('PudID')

In [28]:
z = db_structure.DBExtractor(dataset_name='TOPICC')

In [30]:
z.load_graph()

In [31]:
z.find_paths_between_tables('HOSPITALADMIT', 'DE')

[]

In [19]:
for i in sorted(nx.all_simple_paths(z.g, 'HOSPITALADMIT', 'DEATH'), key=lambda x: len(x)):
    print(i)

['HOSPITALADMIT', 'DEATH']
['HOSPITALADMIT', 'HOSPITALDISCHARGE', 'DEATH']
['HOSPITALADMIT', 'PICUADMIT', 'DEATH']
['HOSPITALADMIT', 'PHYSIOSTATUS', 'DEATH']
['HOSPITALADMIT', 'PICUDISCHARGE', 'DEATH']
['HOSPITALADMIT', 'LIMITOFCARE', 'DEATH']
['HOSPITALADMIT', 'CAREPROCESSES', 'DEATH']
['HOSPITALADMIT', 'HOSPITALDISCHARGE', 'PICUADMIT', 'DEATH']
['HOSPITALADMIT', 'HOSPITALDISCHARGE', 'PHYSIOSTATUS', 'DEATH']
['HOSPITALADMIT', 'HOSPITALDISCHARGE', 'PICUDISCHARGE', 'DEATH']
['HOSPITALADMIT', 'HOSPITALDISCHARGE', 'LIMITOFCARE', 'DEATH']
['HOSPITALADMIT', 'HOSPITALDISCHARGE', 'CAREPROCESSES', 'DEATH']
['HOSPITALADMIT', 'PICUADMIT', 'HOSPITALDISCHARGE', 'DEATH']
['HOSPITALADMIT', 'PICUADMIT', 'PHYSIOSTATUS', 'DEATH']
['HOSPITALADMIT', 'PICUADMIT', 'PICUDISCHARGE', 'DEATH']
['HOSPITALADMIT', 'PICUADMIT', 'LIMITOFCARE', 'DEATH']
['HOSPITALADMIT', 'PICUADMIT', 'CAREPROCESSES', 'DEATH']
['HOSPITALADMIT', 'PHYSIOSTATUS', 'HOSPITALDISCHARGE', 'DEATH']
['HOSPITALADMIT', 'PHYSIOSTATUS', 'PICUADMIT

In [ ]:
path = ['HOSPITALADMIT', 'CAREPROCESSES', 'DEATH']
start = time.time()
df = z.get_df_from_path(path, table_columns_of_interest=[('HOSPITALADMIT', 'PudID'), ('HOSPITALADMIT', 'Sex'), ('CAREPROCESSES', 'MechVent'), ('DEATH', 'DeathMode')])
end = time.time()
print(end-start)

In [ ]:
df

In [ ]:
x = db_structure.DBMaker(dataset_name='SCA_AcuteCare', sql_server='CYKPANADBSQL', sql_db='SCA_AcuteCare', schema_name='dbo')
x.create_db_metadata(ignore_tables_with_substrings=['MOTempRecsGrp', 'MO_MeasureInfo', 'MO_MedicationDim_55_'])

In [ ]:
y = db_structure.DBLinker(dataset_name='SCA_AcuteCare')
for i in y.get_common_column_names():
    if i.upper()[-2:] == 'ID' and i.upper() not in ['GUID']:
        print(i)
        y.add_global_fk(i)

In [ ]:
db_structure.DBDestroyer('SCA_AcuteCare').remove_db()

In [ ]:
z = db_structure.DBExtractor('SCA_AcuteCare')

In [ ]:
paths = [['BF_FoleyTimes', 'SCAVisit']]
table_columns_of_interest = [('BF_FoleyTimes', 'FoleyDays'), ('SCAVisit', 'IsCHF')]
filters_with_name_keys = {'SCAVisit_IsCHF': {'filter': ['0', '1'], 'type': 'bool'}}
groupby_columns = ['SCAVisit_IsCHF']
aggregate_column = 'BF_FoleyTimes_FoleyDays'
aggregate_fxn = 'Mean'
df = z.get_biggest_df_from_paths(paths, table_columns_of_interest, limit_rows=10000)

In [ ]:
agg_df = z.aggregate_df(df, groupby_columns, filters_with_name_keys, aggregate_column, aggregate_fxn)

In [ ]:
agg_df

In [ ]:
import math
math.ceil(50/100*25)

In [ ]:
from web import db
db.session.rollback()

In [ ]:
query = f"SELECT tbl.name, MAX(CAST(p.rows AS int)) AS rows FROM sys.tables AS tbl INNER JOIN sys.indexes AS idx ON idx.object_id = tbl.object_id and idx.index_id < 2 INNER JOIN sys.partitions AS p ON p.object_id=CAST(tbl.object_id AS int) AND p.index_id=idx.index_id WHERE (SCHEMA_NAME(tbl.schema_id)='dbo') GROUP BY tbl.name"
num_rows_df = db_structure.execute_sql_query(query=query, sql_server='CYKPANADBSQL', sql_db='SCA_AcuteCare')

In [ ]:
import math
min_rows=1000
max_rows=10000
analyze_percentage=50
num_rows_in_db = num_rows_df[num_rows_df['name'] == 'CV3ClientVisit_Test'].iloc[0]['rows']
by_percentage = math.ceil(analyze_percentage / 100 * num_rows_in_db)
if by_percentage < min_rows:
    num_rows = min_rows
elif by_percentage > max_rows:
    num_rows = max_rows
else:
    num_rows = by_percentage

In [ ]:
num_rows

In [ ]:
import networkx as nx

In [11]:
g = nx.DiGraph()
g.add_node('A')
g.add_node('B')
g.add_node('C')
g.add_node('D')
g.add_node('E')
g.add_node('F')
#g.add_node('A')
g.add_edge('A', 'C', weight=2)
g.add_edge('A', 'D', weight=4)
g.add_edge('A', 'B', weight=1)
g.add_edge('B', 'A', weight=3)
g.add_edge('B', 'E', weight=5)
g.add_edge('C', 'D', weight=2)
g.add_edge('C', 'F', weight=3)
g.add_edge('D', 'C', weight=3)
g.add_edge('E', 'F', weight=3)

In [12]:
for i in nx.all_pairs_shortest_path(g):
    print(i)

('A', {'A': ['A'], 'C': ['A', 'C'], 'D': ['A', 'D'], 'B': ['A', 'B'], 'F': ['A', 'C', 'F'], 'E': ['A', 'B', 'E']})
('B', {'B': ['B'], 'A': ['B', 'A'], 'E': ['B', 'E'], 'C': ['B', 'A', 'C'], 'D': ['B', 'A', 'D'], 'F': ['B', 'E', 'F']})
('C', {'C': ['C'], 'D': ['C', 'D'], 'F': ['C', 'F']})
('D', {'D': ['D'], 'C': ['D', 'C'], 'F': ['D', 'C', 'F']})
('E', {'E': ['E'], 'F': ['E', 'F']})
('F', {'F': ['F']})
